In [1]:
# Packages
import numpy as np
import sklearn 
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy.integrate import odeint
import math

from utility_code.utility import utils

In [95]:
#import csvs
train = pd.read_csv("ucla2020-cs145-covid19-prediction/train.csv")
test = pd.read_csv("ucla2020-cs145-covid19-prediction/test.csv")
graph = pd.read_csv("ucla2020-cs145-covid19-prediction/graph.csv")
dummydata = pd.read_csv("data-test/raw_data_test.csv", skiprows=2, thousands=',')

In [96]:
train['Population'] = train['Confirmed'].divide(train['Incident_Rate'])*1e5
train['Active/Population'] = train['Active'].divide(train['Population'])
train['Deaths/Population'] = train['Deaths'].divide(train['Population'])
train['Active/People_Tested*Population'] = train['Active']/train['People_Tested']*train['Population']
train['Recovered/Population'] = train['Recovered']/train['Population']


confirmed = train['Confirmed'].to_list()
daily = [confirmed[i]-confirmed[i-50] for i in range(len(confirmed)) if i-50 >= 0]
for i in range(50):
    daily.insert(0,0)

train['New_Cases'] = daily

tests = train['People_Tested'].to_list()
dailyTests = [tests[i]-tests[i-50] for i in range(len(tests)) if i-50 >= 0]
for i in range(50):
    dailyTests.insert(0,0)

train['New_Tests'] = dailyTests

train['New_Cases/New_Tests'] = train['New_Cases']/train['New_Tests']

# population = {}
# for state in states:
#     population[state] = (statesdata[state]['Confirmed'].iloc[0]/ statesdata[state]['Incident_Rate'].iloc[0])*1e5


In [97]:
train

,ID,Province_State,Date,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,...,Testing_Rate,Hospitalization_Rate,Population,Active/Population,Deaths/Population,Active/People_Tested*Population,Recovered/Population,New_Cases,New_Tests,New_Cases/New_Tests
0,0,Alabama,04-12-2020,3563,93,NaN,3470.0,75.988020,21583.0,437.0,...,460.300152,12.264945,4.688897e+06,0.000740,0.000020,7.538559e+05,NaN,0,0.0,NaN
1,1,Alaska,04-12-2020,272,8,66.0,264.0,45.504049,8038.0,31.0,...,1344.711576,11.397059,5.977490e+05,0.000442,0.000013,1.963246e+04,0.000110,0,0.0,NaN
2,2,Arizona,04-12-2020,3542,115,NaN,3427.0,48.662422,42109.0,NaN,...,578.522286,NaN,7.278717e+06,0.000471,0.000016,5.923713e+05,NaN,0,0.0,NaN
3,3,Arkansas,04-12-2020,1280,27,367.0,1253.0,49.439423,19722.0,130.0,...,761.753354,10.156250,2.589027e+06,0.000484,0.000010,1.644889e+05,0.000142,0,0.0,NaN
4,4,California,04-12-2020,22795,640,NaN,22155.0,58.137726,190328.0,5234.0,...,485.423869,22.961176,3.920862e+07,0.000565,0.000016,4.564052e+06,NaN,0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7095,7095,Virginia,08-31-2020,120575,2580,15085.0,102910.0,1412.626461,1586551.0,NaN,...,18587.633628,NaN,8.535519e+06,0.012057,0.000302,5.536477e+05,0.001767,846,13124.0,0.064462
7096,7096,Washington,08-31-2020,74635,1915,NaN,72720.0,980.118828,1461354.0,NaN,...,19190.735838,NaN,7.614893e+06,0.009550,0.000251,3.789328e+05,NaN,315,0.0,inf
7097,7097,West Virginia,08-31-2020,10249,215,8017.0,2017.0,571.883891,436047.0,NaN,...,24330.984010,NaN,1.792147e+06,0.001125,0.000120,8.289841e+03,0.004473,140,4925.0,0.028426
7098,7098,Wisconsin,08-31-2020,75603,1122,67234.0,7247.0,1298.477578,1253422.0,NaN,...,21527.457417,NaN,5.822434e+06,0.001245,0.000193,3.366398e+04,0.011547,266,3820.0,0.069634


In [98]:
statesdata = {}
states = pd.Series.unique(train['Province_State'])
num_states = len(states)
for s in states:
    statesdata[s] = train.loc[train['Province_State'] == s ,:]

In [100]:
for x in states:
    statesdata[x]['Confirmed_diff'] = statesdata[x]['Confirmed'] - statesdata[x]['Confirmed'].shift(1)
    statesdata[x]['Deaths_diff'] = statesdata[x]['Deaths'] - statesdata[x]['Deaths'].shift(1)


C:\Users\shail\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\shail\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [101]:
statesdata['California']

,ID,Province_State,Date,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,...,Population,Active/Population,Deaths/Population,Active/People_Tested*Population,Recovered/Population,New_Cases,New_Tests,New_Cases/New_Tests,Confirmed_diff,Deaths_diff
4,4,California,04-12-2020,22795,640,NaN,22155.0,58.137726,190328.0,5234.0,...,3.920862e+07,0.000565,0.000016,4.564052e+06,NaN,0,0.0,NaN,NaN,NaN
54,54,California,04-13-2020,23931,714,NaN,23217.0,61.035048,190882.0,3015.0,...,3.920862e+07,0.000592,0.000018,4.768949e+06,NaN,1136,554.0,2.050542,1136.0,74.0
104,104,California,04-14-2020,25356,767,NaN,24589.0,64.669453,202208.0,5163.0,...,3.920862e+07,0.000627,0.000020,4.767867e+06,NaN,1425,11326.0,0.125817,1425.0,53.0
154,154,California,04-15-2020,26686,860,NaN,25826.0,68.061564,216486.0,5163.0,...,3.920862e+07,0.000659,0.000022,4.677447e+06,NaN,1330,14278.0,0.093150,1330.0,93.0
204,204,California,04-16-2020,27677,956,NaN,26721.0,70.589069,246400.0,5031.0,...,3.920862e+07,0.000682,0.000024,4.252003e+06,NaN,991,29914.0,0.033128,991.0,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6854,6854,California,08-27-2020,691821,12677,NaN,679144.0,1750.903765,10918415.0,NaN,...,3.951222e+07,0.017188,0.000321,2.457728e+06,NaN,4817,85658.0,0.056235,4817.0,145.0
6904,6904,California,08-28-2020,697385,12805,NaN,684580.0,1764.985483,11010637.0,NaN,...,3.951222e+07,0.017326,0.000324,2.456650e+06,NaN,5564,92222.0,0.060333,5564.0,128.0
6954,6954,California,08-29-2020,702038,12894,NaN,689144.0,1776.761586,11109630.0,NaN,...,3.951222e+07,0.017441,0.000326,2.450992e+06,NaN,4653,98993.0,0.047003,4653.0,89.0
7004,7004,California,08-30-2020,705951,12937,NaN,693014.0,1786.664851,11231829.0,NaN,...,3.951222e+07,0.017539,0.000327,2.437940e+06,NaN,3913,122199.0,0.032022,3913.0,43.0
